## Week 4, Lab 1: Predicting Left-Handedness from Psychological Factors
> Author: Matt Brems

We can sketch out the data science process as follows:
1. Define the problem.
2. Obtain the data.
3. Explore the data.
4. Model the data.
5. Evaluate the model.
6. Answer the problem.

We'll walk through a full data science problem in this lab. 

---
## Step 1: Define The Problem.

You're currently a data scientist working at a university. A professor of psychology is attempting to study the relationship between personalities and left-handedness. They have tasked you with gathering evidence so that they may publish.

Specifically, the professor says "I need to prove that left-handedness is caused by some personality trait. Go find that personality trait and the data to back it up."

As a data scientist, you know that any real data science problem must be **specific** and **conclusively answerable**. For example:
- Bad data science problem: "What is the link between obesity and blood pressure?"
    - This is vague and is not conclusively answerable. That is, two people might look at the conclusion and one may say "Sure, the problem has been answered!" and the other may say "The problem has not yet been answered."
- Good data science problem: "Does an association exist between obesity and blood pressure?"
    - This is more specific and is conclusively answerable. The problem specifically is asking for a "Yes" or "No" answer. Based on that, two independent people should both be able to say either "Yes, the problem has been answered" or "No, the problem has not yet been answered."
- Excellent data science problem: "As obesity increases, how does blood pressure change?"
    - This is very specific and is conclusively answerable. The problem specifically seeks to understand the effect of one variable on the other.

### 1. In the context of the left-handedness and personality example, what are three specific and conclusively answerable problems that you could answer using data science? 

> You might find it helpful to check out the codebook in the repo for some inspiration.

1) If a person playfully insults their friends, how does the probability of being left-handed change?

2) If a person enjoys dancing alone, how does the probability of being left-handed change?

3) If a person hates shopping, how does the probability of being left-handed change?

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error

from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import PolynomialFeatures, OrdinalEncoder

from category_encoders import OneHotEncoder

from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import RFE

---
## Step 2: Obtain the data.

### 2. Read in the file titled "data.csv."
> Hint: Despite being saved as a .csv file, you won't be able to simply `pd.read_csv()` this data!

In [2]:
left_hand = pd.read_csv(filepath_or_buffer = './data.csv', sep='\t')
left_hand.head()

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,...,country,fromgoogle,engnat,age,education,gender,orientation,race,religion,hand
0,4,1,5,1,5,1,5,1,4,1,...,US,2,1,22,3,1,1,3,2,3
1,1,5,1,4,2,5,5,4,1,5,...,CA,2,1,14,1,2,2,6,1,1
2,1,2,1,1,5,4,3,2,1,4,...,NL,2,2,30,4,1,1,1,1,2
3,1,4,1,5,1,4,5,4,3,5,...,US,2,1,18,2,2,5,3,2,2
4,5,1,5,1,5,1,5,1,3,1,...,US,2,1,22,3,1,1,3,2,3


### 3. Suppose that, instead of us giving you this data in a file, you were actually conducting a survey to gather this data yourself. From an ethics/privacy point of view, what are three things you might consider when attempting to gather this data?
> When working with sensitive data like sexual orientation or gender identity, we need to consider how this data could be used if it fell into the wrong hands!

I would keep the following columns hidden/anonymous:
1) Gender

2) Sexual Orientation

3) Religion

---
## Step 3: Explore the data.

### 4. Conduct exploratory data analysis on this dataset.
> If you haven't already, be sure to check out the codebook in the repo, as that will help in your EDA process.

In [3]:
left_hand.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4184 entries, 0 to 4183
Data columns (total 56 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Q1           4184 non-null   int64 
 1   Q2           4184 non-null   int64 
 2   Q3           4184 non-null   int64 
 3   Q4           4184 non-null   int64 
 4   Q5           4184 non-null   int64 
 5   Q6           4184 non-null   int64 
 6   Q7           4184 non-null   int64 
 7   Q8           4184 non-null   int64 
 8   Q9           4184 non-null   int64 
 9   Q10          4184 non-null   int64 
 10  Q11          4184 non-null   int64 
 11  Q12          4184 non-null   int64 
 12  Q13          4184 non-null   int64 
 13  Q14          4184 non-null   int64 
 14  Q15          4184 non-null   int64 
 15  Q16          4184 non-null   int64 
 16  Q17          4184 non-null   int64 
 17  Q18          4184 non-null   int64 
 18  Q19          4184 non-null   int64 
 19  Q20          4184 non-null 

In [4]:
left_hand.describe()

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,...,testelapse,fromgoogle,engnat,age,education,gender,orientation,race,religion,hand
count,4184.000000,4184.000000,4184.000000,4184.000000,4184.000000,4184.000000,4184.000000,4184.000000,4184.000000,4184.000000,...,4184.000000,4184.000000,4184.000000,4184.000000,4184.000000,4184.000000,4184.000000,4184.000000,4184.000000,4184.000000
mean,1.962715,3.829589,2.846558,3.186902,2.865440,3.672084,3.216539,3.184512,2.761233,3.522945,...,479.994503,1.576243,1.239962,30.370698,2.317878,1.654398,1.833413,5.013623,2.394359,1.190966
std,1.360291,1.551683,1.664804,1.476879,1.545798,1.342238,1.490733,1.387382,1.511805,1.242890,...,3142.178542,0.494212,0.440882,367.201726,0.874264,0.640915,1.303454,1.970996,2.184164,0.495357
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,7.000000,1.000000,0.000000,13.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,3.000000,1.000000,2.000000,1.000000,3.000000,2.000000,2.000000,1.000000,3.000000,...,186.000000,1.000000,1.000000,18.000000,2.000000,1.000000,1.000000,5.000000,1.000000,1.000000
50%,1.000000,5.000000,3.000000,3.000000,3.000000,4.000000,3.000000,3.000000,3.000000,4.000000,...,242.000000,2.000000,1.000000,21.000000,2.000000,2.000000,1.000000,6.000000,2.000000,1.000000
75%,3.000000,5.000000,5.000000,5.000000,4.000000,5.000000,5.000000,4.000000,4.000000,5.000000,...,324.250000,2.000000,1.000000,27.000000,3.000000,2.000000,2.000000,6.000000,2.000000,1.000000
max,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,...,119834.000000,2.000000,2.000000,23763.000000,4.000000,3.000000,5.000000,7.000000,7.000000,3.000000


---
## Step 4: Model the data.

### 5. Suppose I wanted to use Q1 - Q44 to predict whether or not the person is left-handed. Would this be a classification or regression problem? Why?

Classification because we are determining the probability between three options. It is a discrete variable we are solving for.

### 6. We want to use $k$-nearest neighbors to predict whether or not a person is left-handed based on their responses to Q1 - Q44. Before doing that, however, you remember that it is often a good idea to standardize your variables. In general, why would we standardize our variables? Give an example of when we would standardize our variables.

Generally, we want to standardize variables when the features have a very wide range of values. The Titanic dataset from a previous lab would be a good example of a dataset we would need to standardize variables because they asked for age, count of siblings, price of ticket, etc. All of these features have different ranges and measurements, so they need to be put on the same scale.

### 7. Give an example of when we might not standardize our variables.

We wouldn't standardize our variables when every feature is on the same scale or measurement.

### 8. Based on your answers to 6 and 7, do you think we should standardize our predictor variables in this case? Why or why not?

No, because questions 1-44 are all based on the same answer scale of 1-5.

### 9. We want to use $k$-nearest neighbors to predict whether or not a person is left-handed. What munging/cleaning do we need to do to our $y$ variable in order to explicitly answer this question? Do it.

There are 11 zeros, which is not associated with a possible answer to the question. Since it's impossible to determine these, I will drop the rows.

In [5]:
left_hand['hand'].value_counts()

1    3542
2     452
3     179
0      11
Name: hand, dtype: int64

In [6]:
hand_cleaned = left_hand[left_hand['hand'] > 0]
hand_cleaned['hand'].value_counts()

1    3542
2     452
3     179
Name: hand, dtype: int64

### 10. The professor for whom you work suggests that you set $k = 4$. In this specific case, why might this be a bad idea?

We shouldn't should an even number for k because if there is a tie, sklearn does not handle it in a logical way. It is always best to use an odd number so there will be a clear winner.cross_val_score(knn, X_train_scaled, y_train)

### 11. Let's *(finally)* use $k$-nearest neighbors to predict whether or not a person is left-handed!

> Be sure to create a train/test split with your data!

> Create four separate models, one with $k = 3$, one with $k = 5$, one with $k = 7$, and one with $k = 9$.

> Instantiate and fit your models using GridSearchCV.

In [7]:
X = hand_cleaned.iloc[:, :44]
y = hand_cleaned['hand']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 8)

In [9]:
params = {'n_neighbors': [3, 5, 7, 9]}

In [10]:
gs = GridSearchCV(KNeighborsClassifier(), params)

In [11]:
gs.fit(X_train, y_train)

GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [3, 5, 7, 9]})

In [12]:
gs.best_params_

{'n_neighbors': 9}

In [13]:
pd.DataFrame(gs.cv_results_).sort_values(by = 'rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
3,0.003125,0.006250,0.087494,0.007652,9,{'n_neighbors': 9},0.848243,0.851438,0.840256,0.848243,0.8448,0.846596,0.003802,1
2,0.009377,0.007656,0.087492,0.018748,7,{'n_neighbors': 7},0.848243,0.849840,0.837061,0.843450,0.8448,0.844679,0.004448,2
1,0.006249,0.007654,0.096868,0.006251,5,{'n_neighbors': 5},0.841853,0.841853,0.830671,0.832268,0.8336,0.836049,0.004829,3
0,0.004105,0.006066,0.084951,0.016863,3,{'n_neighbors': 3},0.835463,0.830671,0.787540,0.816294,0.8064,0.815274,0.017284,4


### 12. How does a null model do? Create a null model. Does it do better than our best KNN model?

In [14]:
hand_cleaned['hand'].value_counts(normalize = True)

1    0.848790
2    0.108315
3    0.042895
Name: hand, dtype: float64

Since the highest percentage of the split is 84.9%, our best KNN model is about the same because the mean test score tells use we are 84.6% accurate.

Being good data scientists, we know that we might not run just one type of model. We might run many different models and see which is best.

### 13. We want to use logistic regression to predict whether or not a person is left-handed. 

Before we do that, let's check the [documentation for logistic regression in sklearn](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html). Is there default regularization? If so, what is it? If not, how do you know?

Yes, regularization is applied by default. The default parameters are penalty='l2' and C=1.0

### 14. We want to use logistic regression to predict whether or not a person is left-handed. Before we do that, should we standardize our features?

No, because they are all on the same 5 point scale.

### 15. Let's use logistic regression to predict whether the person is left-handed.


Be sure to use the same train/test split with your data as with your kNN model above!

Search over the following hyperparameters:

- l2 regularization with C = [.001, .01, .1, 1, 10, 100]


In [15]:
params = {'C': [.001, .01, .1, 1, 10, 100]}
gs2 = GridSearchCV(LogisticRegression(), params, n_jobs = -1)
gs2.fit(X_train, y_train)

C:\Users\Lindsey\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100]})

In [16]:
gs2.best_params_

{'C': 0.001}

In [17]:
pd.DataFrame(gs2.cv_results_).sort_values(by = 'rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.399324,0.048464,0.009798,0.008037,0.001,{'C': 0.001},0.84984,0.849840,0.848243,0.848243,0.8496,0.849153,0.000749,1
1,0.301154,0.011464,0.009374,0.007654,0.01,{'C': 0.01},0.84984,0.848243,0.848243,0.848243,0.8496,0.848834,0.000728,2
2,0.296858,0.009880,0.006250,0.007654,0.1,{'C': 0.1},0.84984,0.848243,0.845048,0.848243,0.8480,0.847875,0.001558,3
3,0.296860,0.013974,0.003123,0.006246,1,{'C': 1},0.84984,0.848243,0.845048,0.848243,0.8480,0.847875,0.001558,3
4,0.287479,0.007655,0.006250,0.007655,10,{'C': 10},0.84984,0.848243,0.845048,0.846645,0.8480,0.847555,0.001613,5
5,0.271856,0.023384,0.000000,0.000000,100,{'C': 100},0.84984,0.848243,0.845048,0.846645,0.8480,0.847555,0.001613,5


---
## Step 5: Evaluate the model(s).

### 16. Before calculating any score on your data, take a step back. 

Think about your $X$ variable and your $Y$ variable. Do you think your $X$ variables will do a good job of predicting your $Y$ variable? Why or why not? What impact do you think this will have on your scores?

I do not think any of the variables used would do a good job predicting the y variable. The questions seem arbitrary and do not relate to what hand a person uses. I think this will have no affect on my scores and the model will randomly guess, being correct the same amount as if someone guessed right-handed every single time.

### 17. Using accuracy as your metric, evaluate the best of your models on both the training (mean validation) and testing sets. Put your scores below. 

In [18]:
knn = KNeighborsClassifier(n_neighbors = 9)
knn.fit(X_train, y_train)
knn.score(X_train, y_train)

0.8497922658996484

In [19]:
knn.score(X_test, y_test)

0.8467432950191571

In [20]:
logr = LogisticRegression(C = 0.001, max_iter = 1000)
logr.fit(X_train, y_train)
logr.score(X_train, y_train)

0.849153084052413

In [21]:
logr.score(X_test, y_test)

0.8477011494252874

### 18. Broadly speaking, how does the value of $k$ in $k$-NN affect the bias-variance tradeoff? (i.e. As $k$ increases, how are bias and variance affected?)

If k is a low number, there is high bias and low variance. As k increases the bias decreases, but then the variance increases.

### 19. If you have a $k$-NN model that has evidence of overfitting, what are three things you might try to do to combat overfitting?

1) Increase k to include more neighbors to compare to.

2) Decrease number of parameters used.

3) Adjust the weights parameter to make the sample more fair.

### 20. Broadly speaking, how does the value of $C$ in logistic regression affect the bias-variance tradeoff? (i.e. As $C$ increases, how are bias and variance affected?)

Increasing the C parameter increases the number of points allowed in the training data,  which increases the bias but decreases the variance.

### 21. For your logistic regression models, play around with the regularization hyperparameter, $C$. As you vary $C$, what happens to the fit and coefficients in the model? What do you think this means in the context of this specific problem?

In [22]:
np.set_printoptions(suppress=True)
logr = LogisticRegression(C = 0.001, max_iter = 1000)
logr.fit(X_train, y_train)
logr.coef_

array([[-0.02647748,  0.00227431, -0.02106437, -0.01572645, -0.01458586,
         0.0146163 , -0.01653068,  0.01775094,  0.00842678,  0.00570285,
        -0.00752813,  0.00563831, -0.00716644,  0.01529073,  0.00851352,
         0.00089831, -0.01539418,  0.02375612, -0.00410477,  0.00731733,
         0.01003156,  0.0229166 ,  0.03392933, -0.00019402, -0.01690437,
        -0.02512919, -0.01148535, -0.02324191, -0.0135265 ,  0.00416453,
        -0.01036909,  0.00309361, -0.01230219, -0.02029108, -0.05055796,
         0.01097466,  0.0102912 , -0.0132055 ,  0.01201209,  0.01601534,
         0.02470312,  0.00078419,  0.00679167, -0.0251883 ],
       [-0.01444559, -0.00353495, -0.01472095, -0.02506887,  0.01304466,
         0.00458398,  0.00319661, -0.04373303, -0.02566687,  0.01929735,
         0.00350209, -0.00970357, -0.01568526, -0.00942392, -0.0082956 ,
        -0.00445964, -0.0025971 , -0.0151911 , -0.00302204, -0.00417585,
         0.01200015, -0.02622951, -0.01819325,  0.00197109,  0.

In [23]:
np.set_printoptions(suppress=True)
logr2 = LogisticRegression(C = 100, max_iter = 1000)
logr2.fit(X_train, y_train)
logr2.coef_

array([[-0.0561403 , -0.00144918, -0.04068638, -0.0493929 , -0.0051921 ,
         0.03050978, -0.02675265,  0.02680686,  0.01571866,  0.03666667,
        -0.02267011,  0.00115206, -0.02248339,  0.04593655,  0.0166111 ,
         0.00165679, -0.02481118,  0.04068063, -0.0185483 ,  0.00511964,
         0.05499295,  0.05030146,  0.07984453, -0.0014914 , -0.03886053,
        -0.04825997, -0.11204434, -0.05597151, -0.00447222,  0.01879691,
        -0.00076147,  0.00610227, -0.04193369, -0.04709328, -0.15108379,
         0.04118014,  0.0222776 , -0.01917264,  0.04960074,  0.00857752,
         0.06179508,  0.00739092,  0.09029712, -0.08350082],
       [-0.06379197, -0.00097435, -0.03647512, -0.09018742,  0.06026943,
         0.03524933,  0.01552488, -0.14501193, -0.0597005 ,  0.1012702 ,
        -0.00911715, -0.02397286, -0.05769045,  0.03435334, -0.00179241,
        -0.00372027,  0.00185711, -0.02624684, -0.01300575, -0.00660276,
        -0.00015287, -0.05013721,  0.00068886,  0.00354734,  0.

In the grid search above, it looks like the mean test score decreases as we increase C. This means that the model works a little bit better when it is more regularized. The test scores also become more varied when increasing C.

This affects the coefficients for the most part by making them go further from zero, meaning negative values are decreasing and positive values are increasing. I think this means my model is overfit when a high C value is used.

### 22. If you have a logistic regression model that has evidence of overfitting, what are three things you might try to do to combat overfitting?

1) Lower the C value.

2) Adjust the penalty to minimize features with collinearity.

3) Decrease number of parameters used.

### 23. How might you deal with the imbalanced dataset?

Add weights to the classes.

---
## Step 6: Answer the problem.

### 24. Suppose you want to understand which psychological features are most important in determining left-handedness. Would you rather use $k$-NN or logistic regression? Why?

Logistic regression because I can evaluate the coefficients easily to see which had the largest impact.

### 25. Instantiate, fit, and score a logistic regression model with no regularization. Interpret the coefficient for `Q1`.

In [24]:
logr3 = LogisticRegression(max_iter = 1000)
logr3.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [25]:
logr3.score(X_train, y_train)

0.849153084052413

In [26]:
logr3.score(X_test, y_test)

0.8477011494252874

In [27]:
logr3.coef_[0][0]

-0.05603626074334529

As the answer for Q1 increases on the scale the probability of which hand the person uses moves in a negative direction, towards right-handed. 

### 26. If you have to select one model overall to be your *best* model, which model would you select? Why?
- Usually in the "real world," you'll fit many types of models but ultimately need to pick only one! (For example, a client may not understand what it means to have multiple models, or if you're using an algorithm to make a decision, it's probably pretty challenging to use two or more algorithms simultaneously.) It's not always an easy choice, but you'll have to make it soon enough. Pick a model and defend why you picked this model!

I chose Logistic Regression with a C value of 0.001. This gave me the highest average score with little variation in score among my test groups.

### 27. Circle back to the three specific and conclusively answerable questions you came up with in Q1. Answer one of these for the professor based on the model you selected!

In [28]:
logr = LogisticRegression(C = 0.001, max_iter = 1000)
logr.fit(X, y)

LogisticRegression(C=0.001, max_iter=1000)

1) If a person playfully insults their friends, how does the probability of being left-handed change?

2) If a person enjoys dancing alone, how does the probability of being left-handed change?

3) If a person hates shopping, how does the probability of being left-handed change?

In [29]:
# Answer to question 1:
logr.coef_[0][22], logr.coef_[1][22], logr.coef_[2][22]

(0.028058293091666277, -0.014567496244366776, -0.013490796847299438)

A person who playfully insults their friends is positively correlated with using their right hand and negatively correlated with using their left hand or both hands.

In [30]:
# Answer to question 2:
logr.coef_[0][13], logr.coef_[1][13], logr.coef_[2][13]

(0.011284472142556222, -0.0015593742753624029, -0.009725097867193838)

A person who enjoys dancing alone is positively correlated with using their right hand and negatively correlated with using their left hand or both hands.

In [31]:
# Answer to question 3:
logr.coef_[0][32], logr.coef_[1][32], logr.coef_[2][32]

(-0.013208192946627572, -0.009401461050986168, 0.022609653997613725)

A person who hates shopping is negatively correlated using their right hand or left hand and positively correlated with using both hands.

### BONUS:
Looking for more to do? Probably not - you're busy! But if you want to, consider exploring the following. (They could make for a blog post!)
- Create a visual plot comparing training and test metrics for various values of $k$ and various regularization schemes in logistic regression.
- Rather than just evaluating models based on accuracy, consider using sensitivity, specificity, etc.
- In the context of predicting left-handedness, why are unbalanced classes concerning? If you were to re-do this process given those concerns, what changes might you make?
